#**12장. 강화 학습**

In [1]:
pip install gym

In [2]:
# 딥 큐-러닝 예제

#라이브러리 호출
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v1').unwrapped #cartpole 강화 학습 환경 불러옴
plt.ion()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
#리플레이 메모리 : 에이전트가 관찰한 상태 전이, 상태, 행동 등 저장 -> 나중에 사용

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward')) #현재상태, 행동, 다음상태, 보상 정보 관리

class ReplayMemory(object) :
  def __init__(self, capacity) :
    self.capacity = capacity
    self.memory = []
    self.position = 0

  def push(self, *args) :
    if len(self.memory) < self.capacity :
      self.memory.append(None)
    self.memory[self.position] = Transition(*args)
    self.position = (self.position + 1) % self.capacity

  def sample(self, batch_size) : #리플레이 메모리에 저장된 데이터 중, 랜덤하게 배치 크기만큼 반환
    return random.sample(self.memory, batch_size)

  def __len__(self) :
    return len(self.memory)

In [6]:
#DQN(Deep Q-Learning) 네트워크

class DQN(nn.Module) :
  def __init__(self, h, w, outputs) :
    super(DQN, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
    self.bn3 = nn.BatchNorm2d(32)

    def conv2d_size_out(size, kernel_size, stride=2) : #합성곱층 출력층 한 차원의 크기 계산
      return (size - (kernel_size-1)-1) // stride + 1

    convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w))) #합성곱 다 거친 특성맵의 가로 길이
    convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h))) #합성곱 다 거친 특성맵의 세로 길이
    linear_input_size = convw * convh * 32 #선형층에 들어가는 크기(가로 * 세로 * 채널)
    self.head = nn.Linear(linear_input_size, outputs)

  def forward(self, x) :
    x = F.relu( self.bn1( self.conv1(x) ) ) #1차 합성곱층 (합성곱 + 배치정규화 + 렐루함수)
    x = F.relu( self.bn2( self.conv2(x) ) ) #2차 합성곱층
    x = F.relu( self.bn3( self.conv3(x) ) ) #3차 합성곱층
    return self.head( x.view(x.size(0), -1) ) #완전연결층 통과하여 리턴

In [7]:
pip install pyglet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 11.9 MB/s eta 0:00:00


In [9]:
#이미지 추출 및 처리
import pyglet

resize = T.Compose( [T.ToPILImage(), T.Resize(40, interpolation=Image.CUBIC), T.ToTensor()] )
          #이미지 크기 및 텐서 변환

def get_cart_location(screen_width) : #카트의 위치 정보 가져오기
  world_width = env.x_threshold * 2
  scale = screen_width / world_width
  return int(env.state[0] * scale + screen_width / 2.0)

def get_screen() :
  screen = env.render(mode='rgb_array').transpose((2, 0, 1))
  _, screen_height, screen_width = screen.shape
  screen = screen[ :, int(screen_height*0.4):int(screen_height * 0.8) ]
  view_width = int(screen_width * 0.6)
  cart_location = get_cart_location(screen_width)

  if cart_location < view_width // 2 :
    slice_range = slice(view_width)
  elif cart_location > (screen_width - view_width // 2) :
    slice_range = slice(-view_width, None)
  else :
    slice_range = slice(cart_location - view_width // 2, cart_location + view_width // 2)
    screen = screen[:, :, slice_range]
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    return resize(screen).unsqueeze(0).to(device)

  env.reset()
  plt.figure()
  plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(), interpolation='none')
  plt.title('화면 예시')
  plt.show()

AttributeError: module 'PIL.Image' has no attribute 'CUBIC'

In [10]:
#모델 객체화 및 손실함수 정의
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape
n_actions = env.action_space.n

policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_divt(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state) :
  global steps_done
  sample = random.random()
  eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
  steps_done += 1
  if sample > eps_threshold :
    with torch.no_grad() :
      return policy_net(state).max(1)[1].view(1, 1)
  else :
    return torch.tensor([ [random.randrange(n_actions)] ], device=device, dtype=torch.long)

episode_durations = []

NameError: name 'get_screen' is not defined

In [ ]:
#모델에서 사용할 옵티마이저 선택

def optimize_model() :
  if len(memory) < BATCH_SIZE :
    return

  transitions = memory.sample(BATCH_SIZE)
  batch = Transition(*zip(*transitions))
  non_final_mask = torch.tensor(tuple( map(lambda s: s is not None, batch.next_state)))
  non_final_states = torch.cat( [s for s in batch.next_state if s is not None] )

  state_batch = torch.cat(batch.state)
  action_batch = torch.cat(batch.action)
  reward_batch = torch.cat(batch.reward)

  state_action_values = policy_net(state_batch).gather(1, action_batch)
  next_state_values = torch.zeros(BATCH_SIZE, device=device)
  next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
  expected_state_action_values = (next_state_values * GAMMA) + reward_batch #기대가치 예상

  loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
  optimizer.zero_grad()
  loss.backward()

  for param in policy_net.parameters() :
    param.grad.data.clamp_(-1, 1)
  optimizer.step()

In [ ]:
#모델 학습
num_episodes = 50

for i_episode in range(num_episodes) :
  env.reset()
  last_screen = get_screen()
  current_screen = get_screen()
  state = current_screen - last_screen

  for t in count() :
    action = select_action(state) #행동 선택 및 실행
    _, reward, done, _ = env.step(action.item())
    rewward = torch.tensor([reward], device=device)

    last_screen = current_screen
    current_screen = get_screen()

    if not done :
      next_state = current_screen - last_screen
    else :
      next_state = None

    memory.push(state, action, next_state, reward) #상태 전이(현 상태, 행동, 담 상태, 보상)를 메모리에 저장
    state = next_state #다음 상태로 이동

    optimize_model()
    if done :
      episode_durations.append(t + 1)
      break

  if i_episode % TARGET_UPDATE == 0 :
    target_net.load_state_dict(policy_net.state_dict())

print('FINISH')
env.render()
env.close()
plt.show()

In [11]:
#몬테카를로 트리 탐색 예제

#보드 생성
boarder = {'1': ' ', '2': ' ', '3': ' ',
           '4': ' ', '5': ' ', '6': ' ',
           '7': ' ', '8': ' ', '9': ' '}

board_keys = []
for key in boarder :
  board_keys.append(key)

In [ ]:
#화면 출력 함수 정의
def visual_Board(board_num) :
  print(board_num['1'] + '|' + board_num['2'] + '|' + board_num['3'])
  print('-+-+-')
  print(board_num['4'] + '|' + board_num['5'] + '|' + board_num['6'])
  print('-+-+-')
  print(board_num['7'] + '|' + board_num['8'] + '|' + board_num['9'])

In [16]:
#플레이어의 보드 이동 위한 함수 정의(메인 함수)

def game() :
  turn = 'X'
  count = 0
  for i in range(8) :
    visual_Board(boarder)
    print('당신 차례입니다' + turn + '어디로 이동할까요?')
    move = input()
    if boarder[move] == ' ':
      boarder[move] = turn
      count += 1
    else :
      print('이미 채워져 있습니다. \n어디로 이동할까요?')

    if count >= 5 :
      if boarder['1'] == boarder['2'] == boarder['3'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['4'] == boarder['5'] == boarder['6'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['7'] == boarder['8'] == boarder['9'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['1'] == boarder['4'] == boarder['7'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['2'] == boarder['5'] == boarder['8'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['3'] == boarder['6'] == boarder['9'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['1'] == boarder['5'] == boarder['9'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break
      elif boarder['3'] == boarder['5'] == boarder['7'] :
        visual_Board(boarder)
        print('\n게임 종료\n')
        print('-------' + turn + '가 승리했습니다-------')
        break

    if count == 9 :
      print('\n게임 종료\n')
      print('동점입니다')

    if turn == 'X' : #움직임이 있을 때마다 플레이어 변경
      turn = 'Y'
    else :
      turn = 'X'

if __name__ == '__main__' :
  game()

X|Y|X
-+-+-
Y|X| 
-+-+-
 | | 
당신 차례입니다X어디로 이동할까요?
1
이미 채워져 있습니다. 
어디로 이동할까요?
X|Y|X
-+-+-
Y|X| 
-+-+-
 | | 
당신 차례입니다Y어디로 이동할까요?
2
이미 채워져 있습니다. 
어디로 이동할까요?
X|Y|X
-+-+-
Y|X| 
-+-+-
 | | 
당신 차례입니다X어디로 이동할까요?


KeyboardInterrupt: Interrupted by user